In [25]:
import pandas as pd
import math
file = "points.xlsx"
data = pd.read_excel(file)
def compute_player_summaries(data):
    """
    Compute player summaries based on the provided data.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A summary dataframe containing computed metrics for each player.
    """
    # Define aggregation functions for the columns of interest
    aggregations = {
        'Date': 'count',
        'Penalty': 'sum',
        'Friend Referrals': 'sum',
        'Own Goals': 'sum',
        'Goals Conceded': 'sum',
        'Goals': ['mean', 'sum'],
        'Total Points': ['mean', 'sum'],
        'MVP': 'sum',
        'SPL Bonus': 'sum'
    }
    
    # Use groupby with multiple aggregation functions
    player_summary = data.groupby('Player').agg(aggregations)
    
    # Flatten hierarchical columns
    player_summary.columns = ['_'.join(col).strip() for col in player_summary.columns.values]
    
    # Rename columns for clarity
    columns_rename = {
        'Date_count': 'Games Played',
        'Penalty_sum': 'Penalties',
        'Friend Referrals_sum': 'Friend Referrals',
        'Own Goals_sum': 'Own Goals',
        'Goals Conceded_sum': 'Goals Conceded',
        'Goals_mean': 'Average Goals/Game',
        'Goals_sum': 'Total Goals',
        'Total Points_mean': 'Average Total Points/Game',
        'Total Points_sum': 'Total Points',
        'MVP_sum': 'MVP',
        'SPL Bonus_sum': 'SPL Bonus'
    }
    player_summary.rename(columns=columns_rename, inplace=True)
    
    # Calculate games won
    games_won = data[data['Team'] == data['Winning Team']].groupby('Player').size()
    player_summary['Games Won'] = games_won
    player_summary['Games Won'].fillna(0, inplace=True)
    
    # Calculate win ratio
    player_summary['Win Ratio'] = player_summary['Games Won'] / player_summary['Games Played']
    
    return player_summary

def calculate_cumulative_points_and_rank(data):
    """
    Calculate cumulative points and rank changes for each player.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A dataframe containing the 'Rank Change' for the latest game for each player.
    """
    # Calculate cumulative Total Points for each player after each game
    data['Cumulative Points'] = data.groupby('Player')['Total Points'].cumsum()
    
    # Determine the player's rank based on these cumulative points after each game
    data['Rank'] = data.groupby('Date')['Cumulative Points'].rank(method="first", ascending=False)
    
    # Sort data to ensure we process in chronological order for each player
    data_sorted = data.sort_values(by=['Player', 'Date'])
    
    # Calculate the change in rank between each game for every player
    data_sorted['Rank Change'] = data_sorted.groupby('Player')['Rank'].diff().fillna(0)
    
    # Extract the latest rank change for each player
    latest_rank_change = data_sorted.groupby('Player').apply(lambda x: x.iloc[-1])['Rank Change']
    
    return latest_rank_change

# Integrate functions to generate the final summary
player_summary = compute_player_summaries(data)
latest_rank_change = calculate_cumulative_points_and_rank(data)

# Add 'Rank Change' to the summary
player_summary['Rank Change'] = latest_rank_change

# Fill NaN values with 0
player_summary = player_summary.fillna(0)

# Convert specific columns to integer type
cols_to_int = ['Games Won', 'MVP', 'SPL Bonus', 'Rank Change']
player_summary[cols_to_int] = player_summary[cols_to_int].astype(int)

# Calculate the overall rank based on the 'Total Points'
player_summary['Overall Rank'] = player_summary['Total Points'].rank(method="min", ascending=False).astype(int)

# Reorder columns to have 'Overall Rank' at the front
column_order = ['Overall Rank'] + [col for col in player_summary if col != 'Overall Rank']
final_summary = player_summary[column_order]

# Sorting the final_summary DataFrame by 'Overall Rank' in ascending order
sorted_summary = final_summary.sort_values(by='Overall Rank')

# Setting option to display all rows
pd.set_option('display.max_rows', None)

# Reordering the columns as specified
desired_column_order = [
    "Overall Rank", "Games Played", "Games Won", "Win Ratio", "Penalties", 
    "Friend Referrals", "Own Goals", "Goals Conceded", "MVP", "SPL Bonus",
    "Average Goals/Game", "Total Goals", "Average Total Points/Game", 
    "Total Points", "Rank Change"
]

sorted_summary = sorted_summary[desired_column_order]
sorted_summary


,Overall Rank,Games Played,Games Won,Win Ratio,Penalties,Friend Referrals,Own Goals,Goals Conceded,MVP,SPL Bonus,Average Goals/Game,Total Goals,Average Total Points/Game,Total Points,Rank Change
Player,,,,,,,,,,,,,,,
Daniele (Francesco),1,17,12,0.705882,0,1,0,108,0,0,3.588235,61,14.470588,246,0
Luca Stoppi,2,23,8,0.347826,1,4,2,166,1,1,0.782609,18,10.652174,245,1
Andrea Limonta,3,22,11,0.500000,0,2,1,154,0,1,0.136364,3,10.500000,231,0
Cameron McAinsh,4,22,9,0.409091,0,1,0,181,0,0,1.818182,40,10.318182,227,-1
Damiano Barbanera,5,21,7,0.333333,0,2,0,161,0,1,2.285714,48,10.571429,222,-2
Giovanni Aiello,6,21,11,0.523810,0,0,0,147,0,2,0.571429,12,10.142857,213,2
Sergio,7,17,9,0.529412,0,2,0,106,2,0,0.705882,12,10.764706,183,2
Federico Paolucci,8,19,9,0.473684,0,0,0,140,1,1,0.263158,5,9.473684,180,-2
Mazzu,9,14,4,0.285714,0,3,1,110,1,2,3.285714,46,12.642857,177,-2


In [26]:
html_file_path = "player_summary.html"
# Convert the final sorted summary into an HTML file, including the "Player" column as the index
sorted_summary.to_html(html_file_path, index=True)

In [27]:
# Parsing the HTML content to extract table data
with open("player_summary.html", "r") as file:
    html_content = file.read()

df = pd.read_html(html_content, header=[0,1], index_col=0)[0]

# First, let's read the content of the provided file to see what we're working with.
with open("modern_table.html", "r") as file:
    modern_table_content = file.read()

# Displaying the first 1000 characters of the file for a quick look
modern_table_content[:1000]

updated_table_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Player Summary</title>
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
            font-family: Arial, sans-serif;
        }
        th, td {
            border: 1px solid #dddddd;
            padding: 8px;
            text-align: left;
        }
        tr:nth-child(even) {
            background-color: #f2f2f2;
        }
        th {
            background-color: #4CAF50;
            color: white;
        }

        /* Responsive styles for mobile screens */
        @media screen and (max-width: 600px) {
            th, td {
                font-size: 0.8em;
                padding: 4px;
            }
        }
    </style>
</head>
<body>
""" + modern_table_content.split("<style>", 1)[-1] + "</body></html>"

In [28]:
# Saving the modernized table to an HTML file
with open("modern_table.html", "w") as file:
    file.write(modern_table_html)